# Embedded ML - Lab 2.2: TensorFlow Lite

In this lab you will learn the basics of TensorFlow Lite, a complement of TensorFlow that allows you to optimize and run models on constrained devices. It provides a much lighter runtime than TensorFlow but it only supports a subset of the tools available in full TensorFlow.

In this lab you might be given some helper functions but you are expected to write most of the code and be able to explain it at a high level of abstraction and also to modify any part of it.

### Learning outcomes


* Explain the basic concepts associated with TensorFlow Lite
* Develop applications following the basic TensorFlow Lite workflow
* Implement post-training quantization using TensorFlow Lite tools

### TensorFlow Lite workflow
After having built a TensorFlow model, you can convert it to the TensorFlow Lite representation. Then you can run it with the TensorFlow Lite interpreter on your development environment before exporting it and copying it to the target device.

To run the model with TensorFlow Lite you should load the model to the TensorFlow Lite interpreter, allocate the input/output tensors, pass the input data and finally run inference. Notice that TensorFlow Lite API calls are different from those of TensorFlow.

In this part of the assignment, you should create and train a simple model (e.g. a one-neuron network) with TensorFlow and then save it. Then follow the TensorFlow Lite workflow until you are able to run inference and validate the outputs.

In [1]:
#### TENSORFLOW BASIC WORKFLOW
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

2025-05-22 23:10:37.529495: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-22 23:10:37.533022: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-22 23:10:37.542202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747955437.556869  132232 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747955437.561179  132232 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747955437.573377  132232 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:

# Create the model for XOR
model = keras.Sequential([
    layers.Dense(2, activation='relu', input_shape=(2,)),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Generate some data
x_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])

# Train the model
y_train = np.array([[0], [1], [1], [0]])
model.fit(x_train, y_train, epochs=500, verbose=0)  # Increased epochs for better training


# Evaluate the model
x_test = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_test = np.array([[0], [1], [1], [0]])
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {accuracy:.4f}")

# Print the predictions
predictions = model.predict(x_test)
print("Predictions:")
for i, pred in enumerate(predictions):
    print(f"Input: {x_test[i]}, Predicted: {pred}, Actual: {y_test[i]}")


# Save the model to a file
model.save('xor_model.keras')


/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-22 23:10:47.932697: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Test accuracy: 0.7500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predictions:
Input: [0 0], Predicted: [0.46291617], Actual: [0]
Input: [0 1], Predicted: [0.45899835], Actual: [1]
Input: [1 0], Predicted: [0.648431], Actual: [1]
Input: [1 1], Predicted: [0.46291617], Actual: [0]


In [3]:
#### TENSORFLOR LITE BASIC WORKFLOW

# Load model of the XOR
model = tf.keras.models.load_model('xor_model.keras')

# Convert model to TF Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


# Save TF Lite model to a file
import pathlib
tflite_model_path = pathlib.Path('xor_model.tflite')
write_file = tflite_model_path.write_bytes(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpuhc1lkea/assets


INFO:tensorflow:Assets written to: /tmp/tmpuhc1lkea/assets


Saved artifact at '/tmp/tmpuhc1lkea'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 2), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  126353262565264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353262568720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353262567184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353262572752: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1747955464.188171  132232 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1747955464.188196  132232 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-05-22 23:11:04.188513: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpuhc1lkea
2025-05-22 23:11:04.188905: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-05-22 23:11:04.188912: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpuhc1lkea
I0000 00:00:1747955464.191325  132232 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-05-22 23:11:04.191761: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-05-22 23:11:04.205627: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpuhc1lkea
2025-05-22 23:11:04.210325: I tensorflow/cc/saved_model/loader.cc:471] SavedModel 

In [ ]:
# Set up input/output tensors
interpreter = tf.lite.Interpreter(model_content=tflite_model)


In [4]:
!pip install ai-edge-litert
from ai_edge_litert.interpreter import Interpreter
interpreter = Interpreter(model_path=tflite_model_path)


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [8]:
#Allocate tensors on the interpreter
interpreter.allocate_tensors()

# Set input values and output memory 
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

test_input = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)

print("TF Lite Predictions:")
def new_func(output_data):
    return output_data[0]

correct_predictions = 0

for i, single_input in enumerate(test_input):
    # Reshape input to match the expected shape (batch size of 1)
    single_input = single_input.reshape(1, -1)
    interpreter.set_tensor(input_details[0]['index'], single_input)

    # Run inference
    interpreter.invoke()

    # Get outputs
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print(f"Input: {single_input[0]}, Predicted: {new_func(np.round(output_data))}, Actual: {y_test[i][0]}")

    # Check if the prediction is correct
    if np.round(output_data) == y_test[i][0]:
        correct_predictions += 1

# Accuracy
accuracy = correct_predictions / len(test_input)
print(f"TF Lite Model Accuracy: {accuracy:.4f}")

    

TF Lite Predictions:
Input: [0. 0.], Predicted: [0.], Actual: 0
Input: [0. 1.], Predicted: [0.], Actual: 1
Input: [1. 0.], Predicted: [1.], Actual: 1
Input: [1. 1.], Predicted: [0.], Actual: 0
TF Lite Model Accuracy: 0.7500


### Vision model with TensorFlow Lite

In this part of the assignment, you should import a small pre-trained model for a vision application that takes at most 1 MB. Then you should follow the TensorFlow Lite workflow until you are able to run inference and obtain the same results as with TensorFlow.

In [39]:
# Load the model ccn_model.keras
model_cnn = tf.keras.models.load_model('cnn_model.keras')
# Convert model to TF Lite
converter_cnn = tf.lite.TFLiteConverter.from_keras_model(model_cnn)
tflite_model_cnn = converter_cnn.convert()

# Save TF Lite model to a file
tflite_model_cnn_path = pathlib.Path('ccn_model.tflite')
write_file_cnn = tflite_model_cnn_path.write_bytes(tflite_model_cnn)

# Set up input/output tensors
interpreter_cnn = Interpreter(model_path=tflite_model_cnn_path)
interpreter_cnn.allocate_tensors()
# Set input values and output memory
input_details_cnn = interpreter_cnn.get_input_details()
output_details_cnn = interpreter_cnn.get_output_details()


INFO:tensorflow:Assets written to: /tmp/tmp47l03m9d/assets


INFO:tensorflow:Assets written to: /tmp/tmp47l03m9d/assets


Saved artifact at '/tmp/tmp47l03m9d'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer_6')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  126353840504464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353840817360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353840818896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353840819472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353840817168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353840820240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353840819664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353840821008: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1747956527.122355  132232 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1747956527.122375  132232 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-05-22 23:28:47.122535: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp47l03m9d
2025-05-22 23:28:47.122927: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-05-22 23:28:47.122934: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp47l03m9d
2025-05-22 23:28:47.126515: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-05-22 23:28:47.149650: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp47l03m9d
2025-05-22 23:28:47.156806: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 34273 microseconds.


In [40]:
#Load a test dataset MNIST Fashion (Only for testing)
mnist = keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_test = x_test / 255.0
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.int32)

# Only use 100 samples for testing
x_test = x_test[:100]
y_test = y_test[:100]

# Free memory of x_train and y_train
del x_train
del y_train

In [41]:
#Show the format of input and output details of the model
print("Input details:", input_details_cnn)
print("Output details:", output_details_cnn)

# Get the input shape
input_shape_cnn = input_details_cnn[0]['shape']
print("Input shape:", input_shape_cnn)

Input details: [{'name': 'serving_default_input_layer_6:0', 'index': 0, 'shape': array([ 1, 28, 28,  1], dtype=int32), 'shape_signature': array([-1, 28, 28,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 17, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Input shape: [ 1 28 28  1]


In [42]:
correct_predictions_cnn = 0
print("TF Lite CNN Predictions:")

def idex_func(output_data):
    return np.argmax(output_data[0])

for i, single_input in enumerate(x_test):
    # Reshape input to match the expected shape (batch size of 1)
    single_input = single_input.reshape(1, 28, 28, 1)
    interpreter_cnn.set_tensor(input_details_cnn[0]['index'], single_input)

    # Run inference
    interpreter_cnn.invoke()

    # Get outputs
    output_data = interpreter_cnn.get_tensor(output_details_cnn[0]['index'])
    print(f"Input: {i}, Predicted: {idex_func(output_data)}, Actual: {y_test[i]}")

    # Check if the prediction is correct
    if idex_func(output_data) == y_test[i]:
        correct_predictions_cnn += 1

# Accuracy
accuracy_cnn = correct_predictions_cnn / len(y_test)
print(f"TF Lite CNN Model Accuracy: {accuracy_cnn:.4f}")

TF Lite CNN Predictions:
Input: 0, Predicted: 9, Actual: 9
Input: 1, Predicted: 2, Actual: 2
Input: 2, Predicted: 1, Actual: 1
Input: 3, Predicted: 1, Actual: 1
Input: 4, Predicted: 6, Actual: 6
Input: 5, Predicted: 1, Actual: 1
Input: 6, Predicted: 4, Actual: 4
Input: 7, Predicted: 4, Actual: 6
Input: 8, Predicted: 5, Actual: 5
Input: 9, Predicted: 7, Actual: 7
Input: 10, Predicted: 4, Actual: 4
Input: 11, Predicted: 5, Actual: 5
Input: 12, Predicted: 7, Actual: 7
Input: 13, Predicted: 3, Actual: 3
Input: 14, Predicted: 4, Actual: 4
Input: 15, Predicted: 1, Actual: 1
Input: 16, Predicted: 2, Actual: 2
Input: 17, Predicted: 4, Actual: 4
Input: 18, Predicted: 8, Actual: 8
Input: 19, Predicted: 0, Actual: 0
Input: 20, Predicted: 2, Actual: 2
Input: 21, Predicted: 7, Actual: 5
Input: 22, Predicted: 7, Actual: 7
Input: 23, Predicted: 5, Actual: 9
Input: 24, Predicted: 1, Actual: 1
Input: 25, Predicted: 4, Actual: 4
Input: 26, Predicted: 4, Actual: 6
Input: 27, Predicted: 0, Actual: 0
Input

### Post-training quantization
Finally, in this part of the assignment you should activate quantization and convert the model again. Compare model size and accuracy of the compressed TensorFlow Lite model by using various configurations (investigate how) and against the uncompressed baseline.

In [ ]:
# Apply post-training quantization with INT8
converter_cnn.optimizations = [tf.lite.Optimize.DEFAULT]
converter_cnn.representative_dataset = lambda: (tf.data.Dataset.from_tensor_slices(x_test.astype(np.float32))
                                                  .batch(1))
converter_cnn.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
quantized_tflite_model_cnn = converter_cnn.convert()

# Save the quantized model to a file
quantized_tflite_model_cnn_path = pathlib.Path('quantized_cnn_model.tflite')
quantized_write_file_cnn = quantized_tflite_model_cnn_path.write_bytes(quantized_tflite_model_cnn)

# Compare model sizes
print(f"Uncompressed model size: {write_file_cnn} bytes")
print(f"Quantized model size: {quantized_write_file_cnn} bytes")

# Load the quantized model
quantized_interpreter_cnn = Interpreter(model_path=quantized_tflite_model_cnn_path)
quantized_interpreter_cnn.allocate_tensors()

# Set input/output tensors for the quantized model
quantized_input_details_cnn = quantized_interpreter_cnn.get_input_details()
quantized_output_details_cnn = quantized_interpreter_cnn.get_output_details()

# Evaluate the quantized model's accuracy
correct_predictions_quantized_cnn = 0
print("Quantized TF Lite CNN Predictions:")

for i, single_input in enumerate(x_test):
    # Reshape input to match the expected shape (batch size of 1)
    single_input = single_input.reshape(1, 28, 28, 1)
    # Ensure input is quantized to match the model's expected input type
    input_scale, input_zero_point = quantized_input_details_cnn[0]['quantization']
    single_input_quantized = (single_input / input_scale + input_zero_point).astype(quantized_input_details_cnn[0]['dtype'])
    quantized_interpreter_cnn.set_tensor(quantized_input_details_cnn[0]['index'], single_input_quantized)

    # Run inference
    quantized_interpreter_cnn.invoke()

    # Get outputs
    output_data = quantized_interpreter_cnn.get_tensor(quantized_output_details_cnn[0]['index'])
    print(f"Input: {i}, Predicted: {idex_func(output_data)}, Actual: {y_test[i]}")

    # Check if the prediction is correct
    if idex_func(output_data) == y_test[i]:
        correct_predictions_quantized_cnn += 1

# Quantized model accuracy
accuracy_quantized_cnn = correct_predictions_quantized_cnn / len(y_test)
print(f"Quantized TF Lite CNN Model Accuracy: {accuracy_quantized_cnn:.4f}")

INFO:tensorflow:Assets written to: /tmp/tmpx1_a9y8t/assets


INFO:tensorflow:Assets written to: /tmp/tmpx1_a9y8t/assets


Saved artifact at '/tmp/tmpx1_a9y8t'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer_6')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  126353840504464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353840817360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353840818896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353840819472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353840817168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353840820240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353840819664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126353840821008: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/python/3.12.1/lib/python3.12/site-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1747957520.918208  132232 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1747957520.918226  132232 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-05-22 23:45:20.918371: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpx1_a9y8t
2025-05-22 23:45:20.918841: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-05-22 23:45:20.918848: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpx1_a9y8t
2025-05-22 23:45:20.923942: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-05-22 23:45:20.949448: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tm

ValueError: You need to provide either a dictionary with input names and values, a tuple with signature key and a dictionary with input names and values, or an array with input values in the order of input tensors of the graph in the representative_dataset function. Unsupported value from dataset: [[[0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.       